## Initialization

In [1]:
import sys
#sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('I:/내 드라이브/backup/document/USC/Research/MCPC/Code/Codes')

#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

#import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
#from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

#import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

import scipy

import igraph 
import networkx as nx


import metric as met 
import embedding as embed   
import data_utils_ch as data_util

%load_ext autoreload
%autoreload 2
import importlib

import metric as met 
import embedding as embed   
import data_utils_ch as data_util
from subgraph import *


met=importlib.reload(met)
embed=importlib.reload(embed)
data_util=importlib.reload(data_util)

## Testing every dataset with top k % nodes induced subgraph (Louvain)

### Top k% induced subgraph + Strong Majority Vote later

##### PDF Export version (Final Working Version)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import time 
import debug
from collections import defaultdict
import leidenalg as la 
import igraph as ig

res_=[] #0.26, 0.24 is best for Louvain, 3.3, 2.0
labels=[]
total_labels=[]
names = []

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
data_names = ['Zhengmix8eq']
#data_names = ['breast-cancer']
'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''
#data_names = ['Cora full', 'Citeseer', 'Eu core']
# FR_ord = ['Rd', 'Ord']
# FR_Recalc = ['FR_avg', 'FR_Re']
FR_tp = ['FL','FL_ng','FL_ng_prop','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    pdf_name = './Plots/'+data+'/Effective_improv_check'+data+'.pdf'
    
    
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''
        #DO not recalculate Graph for non bulk RNA datasets
        if data not in ['mRNA','miRNA'] and survive==1:
            continue
        G, label= data_to_graph(data,survive)
        cluster_sizes = defaultdict(int)
        for node in G.nodes():
            cluster_sizes[label[node]]+=1
        #print('Selected Labels:',selected_labels_dict)
        #add self loop to nodes without outgoing edges
        #G_ is copy of G
        G_ = G.copy()
        # if (data in ['Cora full', 'Citeseer', 'Eu core']):
        #     for node in G.nodes():
        #         if not list(G.successors(node)):
        #             G_.remove_node(node)
        for node in G_.nodes():
            if not list(G_.successors(node)):
                #print(data, ' has a node without outgoing edges:',node)
                G_.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())
        

        data_directory = f'./Plots/{data}'
        file_path = f'{data_directory}/{data}.txt'
        os.makedirs(data_directory, exist_ok=True)
        with open(file_path, 'w') as f:
            f.write('------------------------------------------------------------------------\n')
        
        
            
        for Num_hop in [1]:
            plt.figure(figsize=(15,15)) # figure for NMI vs purity
            plt.figure(figsize=(15,8)) # figure for NMI vs Purity (whole)
            plt.figure(figsize=(15,15)) # figure for NMI
            plt.figure(figsize=(15,15)) # figure for Purity
            plt.figure(figsize=(15,15)) # figure for preserve_ratio
            plt.figure(figsize=(15,15)) # figure for balancedness
            plt.figure(figsize=(10,15)) # figure for InEdge Degrees
            #List of FR_values
            FR_precalculated = []
            for fr_type in range(4): 
                '''
                Calculate FlowRank for the whole graph
                '''
                if Num_hop==0:
                    node2FR = calc_FlowRank(G_, fr_type, 5)
                else:
                    node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
                FR_precalculated.append(node2FR)

            for idx, res in enumerate([0.05,0.25, 0.5, 1, 1.5, 5]):
            #for idx, res in enumerate([0.05]):
            #for idx, res in enumerate([0.01,0.03,0.05,0.1,0.15,0.2]):
                for Mod_type in [0]:
                    ''' 
                    Louvain on G (baseline)
                    '''
                    partition = debug.louvain_partitions(G, seed=0,resolution=res)
                    partition_ = deque(partition, maxlen=1).pop()
                    label_ = part_to_full_label(partition_,G.number_of_nodes())
                    NMI_baseline = NMI(label_, label)
                    Purity_baseline = met.purity_score(label, label_)
                    
                    '''
                    Leiden on G (Baseline)
                    '''
                    G_ig, mapping = networkx_to_igraph(G) # Mapping = igraph index to networkx index
                    partition = la.find_partition(G_ig, la.RBConfigurationVertexPartition, n_iterations=-1, resolution_parameter=res, seed=0)
                    label_ = part_to_full_label(partition, G.number_of_nodes(), mapping)
                    NMI_baseline_leiden = NMI(label, label_)
                    Purity_baseline_leiden = met.purity_score(label, label_)
                    print('NMI baseline Leiden:', round(NMI_baseline_leiden,3), 'Purity baseline Leiden:', round(Purity_baseline_leiden,3))
                    
                    for FR_type in range(4):
                        name =  data + ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops'
                        if FR_type==3:
                            name = data + ' (top ' + str(k*100) + '%): PageRank'
                        if data in ['mRNA','miRNA']:
                            name = data +  ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops' + ' + Survive: '+str(survive)
                            if FR_type==3:
                                name = data + ' (top ' + str(k*100) + '%): PageRank' + ' + Survive: '+str(survive)
                        print(' ---',name,'| res: ',str(res))
                        
                        # Create the directory if it doesn't exist
                        os.makedirs(data_directory, exist_ok=True)
                        with open(file_path, 'a') as f:
                            f.write(' ---'+name+'| res: '+str(res)+'\n')
                            f.write('NMI baseline: ' + str(round(NMI_baseline,3)) + '\n')
                        

                        '''
                        Get induced subgraph of top k% nodes in FR value
                        '''
                        node2FR = FR_precalculated[FR_type]
                        
                        # if data in ['Cora full', 'Citeseer', 'Eu core']:
                        #     for node in G.nodes():
                        #         if not list(G.successors(node)):
                        #             node2FR[node]=0

                        node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                        
                        k_ = int(k*len(node_list))
                        top_nodes = node_list[:k_]
                        nodes_rest = node_list[k_:]
                        H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                        
                        '''
                        label counts for balancedness calculation
                        '''
                        selected_labels_dict = defaultdict(int)
                        for node in H.nodes():
                            selected_labels_dict[label[node]]+=1
                        
                        '''
                        Louvain on the subgraph H 
                        '''
                        
                        partition = debug.louvain_partitions(H, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        H_label = part_to_full_label(partition_,G.number_of_nodes())

                        '''
                        Add nodes to the subgraph by strong majority vote
                        '''
                        NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes,k)
                        '''
                        Choose same amount of nodes from the list of nodes sorted by FR value and check Louvain NMI and purity
                        '''
                        node_num = len(NMI_List)+H.number_of_nodes()
                        H_ = getInducedSubgraph(G, node_list[:node_num])
                        partition = debug.louvain_partitions(H_, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        H_label_ = part_to_full_label(partition_,G.number_of_nodes())
                        H_label_compressed = []
                        label_compressed = []
                        for i in node_list[:node_num]:
                            if H_label_[i] != -1:
                                H_label_compressed.append(H_label_[i])
                                label_compressed.append(label[i])
                            
                        NMI_majority_vote_percentage_top_nodes = NMI(H_label_compressed, label_compressed)
                        Purity_majority_vote_percentage_top_nodes = met.purity_score(label_compressed, H_label_compressed)
                        print('NMI of top majority vote percentage nodes:', round(NMI_majority_vote_percentage_top_nodes,3), 
                              'Purity of top majority vote percentage nodes:', round(Purity_majority_vote_percentage_top_nodes,3))
                        
                        '''
                        Plots
                        '''
                        NMI_vs_Purity, NMI_vs_Purity_whole, nmi_fig, purity_fig, preserve_fig, balance_fig, InEdge_fig = plt.get_fignums()[-7:]
                        max_dot_size = 100
                        #plot NMI vs Purity
                        markers = ['o','s','*','^','h','D'] #idx (resolution)
                        colors = ['b','g','y','r'] #FR_type
                        plt.figure(NMI_vs_Purity)
                        plt.subplot(3,2,idx+1)
                        percent_nodes = (H.number_of_nodes() + len(NMI_List))*100 / G.number_of_nodes()
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size,label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '% nodes')
                        plt.scatter(Purity_majority_vote_percentage_top_nodes, NMI_majority_vote_percentage_top_nodes, s = percent_nodes/100*max_dot_size, edgecolors = colors[FR_type], facecolors='none', label = 'exp2|' + str(FR_tp[FR_type]) + '|'+str(round(percent_nodes,2))+ '% nodes')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size,label = 'Louv_baseline')
                            plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size,label = 'Leiden_baseline')
                        
                        #plot NMI vs Purity (whole)
                        plt.figure(NMI_vs_Purity_whole)
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size, marker = markers[idx],label = str(FR_tp[FR_type])+ '|'+str(res)+'|'+str(round(percent_nodes,2))+ '% nodes')
                        plt.scatter(Purity_majority_vote_percentage_top_nodes, NMI_majority_vote_percentage_top_nodes , edgecolors = colors[FR_type], facecolors='none', s = percent_nodes/100*max_dot_size, marker = markers[idx],label = 'Exp2|'+str(res)+'|'+str(round(percent_nodes,2))+ '% nodes')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size, marker = markers[idx],label = 'Louv_baseline|'+str(res))
                            plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size, marker = markers[idx],label = 'Leiden_baseline|'+str(res))
                        #plt.scatter(Purity_twice, NMI_twice, color = 'm', s = max_dot_size, marker = markers[idx],label = '2*Louv|'+str(res))

                        #plot NMI
                        plt.figure(nmi_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(NMI_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        NMI_louv_after_vote_list = [NMI_majority_vote_percentage_top_nodes]*len(nodes_rest)
                        plt.plot(NMI_louv_after_vote_list, label = 'Exp2|'+str(FR_tp_short[FR_type]))
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            NMI_baseline_list = [NMI_baseline]*len(nodes_rest)
                            plt.plot(NMI_baseline_list, label = 'Louv_baseline')
                            NMI_baseline_leiden_list = [NMI_baseline_leiden]*len(nodes_rest)
                            plt.plot(NMI_baseline_leiden_list, label = 'Leiden_baseline')
                            
                        #NMI_Louv_twice_list = [NMI_twice]*len(nodes_rest)
                        #plt.plot(NMI_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            f.write('# of nodes not included: ' + str(G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) + '\n')
                            #f.write('NMI_Louvain_twice: ' + str(round(NMI_twice,3)) + '\n')
                            f.write('NMI after top ' + str(k*100) + '% nodes: ' + str(round(NMI_List[0],3)) + '\n')
                            f.write('Max NMI: ' + str(round(max(NMI_List),3)) + '\n')
                        print('Number of nodes not included: ' , G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) 
                        print('NMI after top ', k*100, '% nodes:', round(NMI_List[0],3))
                        print('Max NMI: ', round(max(NMI_List),3))

                        #plot Purity
                        plt.figure(purity_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Purity_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        Purity_Louv_after_vote_list = [Purity_majority_vote_percentage_top_nodes]*len(nodes_rest)
                        plt.plot(Purity_Louv_after_vote_list, label = 'Exp2|'+str(FR_tp_short[FR_type]))
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            Purity_baseline_list = [Purity_baseline]*len(nodes_rest)
                            plt.plot(Purity_baseline_list, label = 'Louv_baseline')
                            Purity_baseline_leiden_list = [Purity_baseline_leiden]*len(nodes_rest)
                            plt.plot(Purity_baseline_leiden_list, label = 'Leiden_baseline')
                            
                    
                        #Purity_Louv_twice_list = [Purity_twice]*len(nodes_rest)
                        #plt.plot(Purity_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            #f.write('Purity_Louvain_twice: ' + str(round(Purity_twice,3)) + '\n')
                            f.write('Purity after top ' + str(k*100) + '% nodes: ' + str(round(Purity_List[0],3)) + '\n')
                            f.write('Max Purity: ' + str(round(max(Purity_List),3)) + '\n')

                        #plot preserve_ratio
                        plt.figure(preserve_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Preserv_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        
                        #plot balancedness
                        plt.figure(balance_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Balance_List,  color = colors[FR_type], label = str(FR_tp[FR_type])) 
                       
                        #plot InEdge Degrees
                        plt.figure(InEdge_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(InEdge_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                    
                plt.figure(NMI_vs_Purity)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3,2,idx+1)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                


                plt.figure(NMI_vs_Purity_whole)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]))
                plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=11, ncol=2)
                plt.tight_layout(rect=[0, 0, 1, 1])

                plt.figure(nmi_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)    
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(purity_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Purity')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(preserve_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Preserve Ratio')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(balance_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Balancedness')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(InEdge_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('InEdge Degrees')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()
    
    
    p = PdfPages(pdf_name)            
    fig_nums = plt.get_fignums() 
    figs = [plt.figure(n) for n in fig_nums] 
	
	# iterating over the numbers in list 
    for fig in figs: 
		# and saving the files 
        fig.savefig(p, format='pdf') 
	
	# close the object 
    p.close() 
    plt.close('all')
    
        
            

                                
                    

                    
                    



Zhengmix8eq 3994
Log transform done
(3994, 50)
Accuracy of  15 -NN graph is 0.794
59910
Number of nodes: 3994
NMI baseline Leiden: 0.688 Purity baseline Leiden: 0.546
 --- Zhengmix8eq (top 20.0%): FL + log(n) hops | res:  0.05
NMI of top majority vote percentage nodes: 0.689 Purity of top majority vote percentage nodes: 0.551
Number of nodes not included:  37
NMI after top  20.0 % nodes: 0.873
Max NMI:  0.876
 --- Zhengmix8eq (top 20.0%): FL_ng + log(n) hops | res:  0.05
NMI of top majority vote percentage nodes: 0.69 Purity of top majority vote percentage nodes: 0.55
Number of nodes not included:  57
NMI after top  20.0 % nodes: 0.753
Max NMI:  0.766
 --- Zhengmix8eq (top 20.0%): FL_ng_prop + log(n) hops | res:  0.05
NMI of top majority vote percentage nodes: 0.689 Purity of top majority vote percentage nodes: 0.551
Number of nodes not included:  37
NMI after top  20.0 % nodes: 0.747
Max NMI:  0.763
 --- Zhengmix8eq (top 20.0%): PageRank | res:  0.05
NMI of top majority vote percentag

## Effective Clustering Improvement Checker

In [35]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import time 
import debug
from collections import defaultdict
import leidenalg as la 
import igraph as ig

res_=[] #0.26, 0.24 is best for Louvain, 3.3, 2.0
labels=[]
total_labels=[]
names = []

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['Zhengmix8eq']
#data_names = ['breast-cancer']
'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''
#data_names = ['Cora full', 'Citeseer', 'Eu core']
# FR_ord = ['Rd', 'Ord']
# FR_Recalc = ['FR_avg', 'FR_Re']
FR_tp = ['FL','FL_ng','FL_ng_prop','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    pdf_name = './Plots/'+data+'/Effective_improv_check'+data+'.pdf'
    
    
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''
        #DO not recalculate Graph for non bulk RNA datasets
        if data not in ['mRNA','miRNA'] and survive==1:
            continue
        G, label= data_to_graph(data,survive)
        cluster_sizes = defaultdict(int)
        for node in G.nodes():
            cluster_sizes[label[node]]+=1
        #print('Selected Labels:',selected_labels_dict)
        #add self loop to nodes without outgoing edges
        #G_ is copy of G
        G_ = G.copy()
        # if (data in ['Cora full', 'Citeseer', 'Eu core']):
        #     for node in G.nodes():
        #         if not list(G.successors(node)):
        #             G_.remove_node(node)
        for node in G_.nodes():
            if not list(G_.successors(node)):
                #print(data, ' has a node without outgoing edges:',node)
                G_.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())
        

        data_directory = f'./Plots/{data}'
        file_path = f'{data_directory}/{data}.txt'
        os.makedirs(data_directory, exist_ok=True)
        with open(file_path, 'w') as f:
            f.write('------------------------------------------------------------------------\n')
        
        
            
        for Num_hop in [1]:
            plt.figure(figsize=(15,15)) # figure for NMI vs purity
            plt.figure(figsize=(15,8)) # figure for NMI vs Purity (whole)
            plt.figure(figsize=(15,15)) # figure for NMI
            plt.figure(figsize=(15,15)) # figure for Purity
            plt.figure(figsize=(15,15)) # figure for preserve_ratio
            plt.figure(figsize=(15,15)) # figure for balancedness
            plt.figure(figsize=(10,15)) # figure for InEdge Degrees
            #List of FR_values
            FR_precalculated = []
            for fr_type in range(4): 
                '''
                Calculate FlowRank for the whole graph
                '''
                if Num_hop==0:
                    node2FR = calc_FlowRank(G_, fr_type, 5)
                else:
                    node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
                FR_precalculated.append(node2FR)

            for idx, res in enumerate([0.05,0.25, 0.5, 1, 1.5, 5]):
            #for idx, res in enumerate([0.05]):
            #for idx, res in enumerate([0.01,0.03,0.05,0.1,0.15,0.2]):
                for Mod_type in [0]:
                    ''' 
                    Louvain on G (baseline)
                    '''
                    partition = debug.louvain_partitions(G, seed=0,resolution=res)
                    partition_ = deque(partition, maxlen=1).pop()
                    label_ = part_to_full_label(partition_,G.number_of_nodes())
                    NMI_baseline = NMI(label_, label)
                    Purity_baseline = met.purity_score(label, label_)
                    
                    '''
                    Leiden on G (Baseline)
                    '''
                    G_ig, mapping = networkx_to_igraph(G) # Mapping = igraph index to networkx index
                    partition = la.find_partition(G_ig, la.RBConfigurationVertexPartition, n_iterations=-1, resolution_parameter=res, seed=0)
                    label_ = part_to_full_label(partition, G.number_of_nodes(), mapping)
                    NMI_baseline_leiden = NMI(label, label_)
                    Purity_baseline_leiden = met.purity_score(label, label_)
                    print('NMI baseline Leiden:', round(NMI_baseline_leiden,3), 'Purity baseline Leiden:', round(Purity_baseline_leiden,3))
                    
                    for FR_type in range(4):
                        name =  data + ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops'
                        if FR_type==3:
                            name = data + ' (top ' + str(k*100) + '%): PageRank'
                        if data in ['mRNA','miRNA']:
                            name = data +  ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops' + ' + Survive: '+str(survive)
                            if FR_type==3:
                                name = data + ' (top ' + str(k*100) + '%): PageRank' + ' + Survive: '+str(survive)
                        print(' ---',name,'| res: ',str(res))
                        
                        # Create the directory if it doesn't exist
                        os.makedirs(data_directory, exist_ok=True)
                        with open(file_path, 'a') as f:
                            f.write(' ---'+name+'| res: '+str(res)+'\n')
                            f.write('NMI baseline: ' + str(round(NMI_baseline,3)) + '\n')
                        

                        '''
                        Get induced subgraph of top k% nodes in FR value
                        '''
                        node2FR = FR_precalculated[FR_type]
                        
                        # if data in ['Cora full', 'Citeseer', 'Eu core']:
                        #     for node in G.nodes():
                        #         if not list(G.successors(node)):
                        #             node2FR[node]=0

                        node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                        
                        k_ = int(k*len(node_list))
                        top_nodes = node_list[:k_]
                        nodes_rest = node_list[k_:]
                        H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                        
                        '''
                        label counts for balancedness calculation
                        '''
                        selected_labels_dict = defaultdict(int)
                        for node in H.nodes():
                            selected_labels_dict[label[node]]+=1
                        
                        '''
                        Louvain on the subgraph H 
                        '''
                        
                        partition = debug.louvain_partitions(H, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        H_label = part_to_full_label(partition_,G.number_of_nodes())

                        '''
                        Add nodes to the subgraph by strong majority vote
                        '''
                        NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                        #The nmi and purity of strong majority vote
                        print('NMI of strong majority vote:', round(NMI_List[-1],3), 'Purity of strong majority vote:', round(Purity_List[-1],3))
                        '''
                        Effective Clustering Improvement Checker
                        '''
                        NMI_, Purity_ = effective_cluster_accuracy(G,label,selected_labels_dict, res)
                        print('NMI of effective cluster accuracy:', round(NMI_,3), 'Purity of effective cluster accuracy:', round(Purity_,3))

                        
                        # '''Run Louvain Again after majority vote'''
                        # init_part = label_to_partition(H_label) #change label (list) to partition (list of sets)
                        # print('Number of clusters before majority vote:', len(set(H_label)))
                        # nodes_selected = []
                        # for idx_, label_ in enumerate(H_label):
                        #     if label_ != -1:
                        #         nodes_selected.append(idx_)
                        # H_ = getInducedSubgraph(G, nodes_selected)
                        # print('Number of nodes in the subgraph:', H_.number_of_nodes())
                        # part = Cust.louvain_partitions(H_, seed=0,resolution=res,init_part=init_part)
                        # part_ = deque(part, maxlen=1).pop()
                        # #compressed labels
                        # H_label_ = part_to_full_label(part_,G.number_of_nodes())
                        # label_compressed, H_label_compressed = get_compressed_labels(H_label_, label, nodes_selected)
                        # # # of clusters 
                        # print('Number of clusters after majority vote:', len(set(H_label_compressed)))
                        # NMI_louv_after_vote = NMI(label_compressed, H_label_compressed)
                        # Purity_louv_after_vote = met.purity_score(label_compressed, H_label_compressed)
                        # print('NMI of Louvain after majority vote:', round(NMI_louv_after_vote,3), 'Purity of Louvain after majority vote:', round(Purity_louv_after_vote,3))
                        '''
                        Plots
                        '''
                        NMI_vs_Purity, NMI_vs_Purity_whole, nmi_fig, purity_fig, preserve_fig, balance_fig, InEdge_fig = plt.get_fignums()[-7:]
                        max_dot_size = 100
                        #plot NMI vs Purity
                        markers = ['o','s','*','^','h','D'] #idx (resolution)
                        colors = ['b','g','y','r'] #FR_type
                        plt.figure(NMI_vs_Purity)
                        plt.subplot(3,2,idx+1)
                        percent_nodes = (H.number_of_nodes() + len(NMI_List))*100 / G.number_of_nodes()
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size,label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '% nodes')
                        plt.scatter(Purity_, NMI_, s = percent_nodes/100*max_dot_size, edgecolors = colors[FR_type], facecolors='none', label = 'Eff_checker|'+str(round(percent_nodes,2))+ '% nodes')
                        #plt.scatter(Purity_louv_after_vote, NMI_louv_after_vote, s = percent_nodes/100*max_dot_size, color = colors[FR_type],marker = '*', label = 'Louv_after_vote|'+str(round(percent_nodes,2))+ '% nodes')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size,label = 'Louv_baseline')
                            plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size,label = 'Leiden_baseline')
                        
                        #plot NMI vs Purity (whole)
                        plt.figure(NMI_vs_Purity_whole)
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size, marker = markers[idx],label = str(FR_tp[FR_type])+ '|'+str(res)+'|'+str(round(percent_nodes,2))+ '% nodes')
                        plt.scatter(Purity_, NMI_, s = percent_nodes/100*max_dot_size, edgecolors = colors[FR_type], facecolors='none', marker = markers[idx], label = 'Eff_checker|' + str(FR_tp[FR_type]) + '|'+str(round(percent_nodes,2))+ '% nodes')
                        #plt.scatter(Purity_louv_after_vte, NMI_louv_after_vote, s = percent_nodes/100*max_dot_size, color = colors[FR_type],marker =markers[idx], label = 'Louv_after_vote|' + str(FR_tp[FR_type]) + '|'+str(round(percent_nodes,2))+ '% nodes')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size, marker = markers[idx],label = 'Louv_baseline|'+str(res))
                            plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size, marker = markers[idx],label = 'Leiden_baseline|'+str(res))
                        #plt.scatter(Purity_twice, NMI_twice, color = 'm', s = max_dot_size, marker = markers[idx],label = '2*Louv|'+str(res))

                        #plot NMI
                        plt.figure(nmi_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(NMI_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            NMI_baseline_list = [NMI_baseline]*len(nodes_rest)
                            plt.plot(NMI_baseline_list, label = 'Louv_baseline')
                            NMI_baseline_leiden_list = [NMI_baseline_leiden]*len(nodes_rest)
                            plt.plot(NMI_baseline_leiden_list, label = 'Leiden_baseline')
                            
                        #NMI_Louv_twice_list = [NMI_twice]*len(nodes_rest)
                        #plt.plot(NMI_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            f.write('# of nodes not included: ' + str(G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) + '\n')
                            #f.write('NMI_Louvain_twice: ' + str(round(NMI_twice,3)) + '\n')
                            f.write('NMI after top ' + str(k*100) + '% nodes: ' + str(round(NMI_List[0],3)) + '\n')
                            f.write('Max NMI: ' + str(round(max(NMI_List),3)) + '\n')
                        print('Number of nodes not included: ' , G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) 
                        print('NMI after top ', k*100, '% nodes:', round(NMI_List[0],3))
                        print('Max NMI: ', round(max(NMI_List),3))

                        #plot Purity
                        plt.figure(purity_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Purity_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            Purity_baseline_list = [Purity_baseline]*len(nodes_rest)
                            plt.plot(Purity_baseline_list, label = 'Louv_baseline')
                            Purity_baseline_leiden_list = [Purity_baseline_leiden]*len(nodes_rest)
                            plt.plot(Purity_baseline_leiden_list, label = 'Leiden_baseline')
                            
                    
                        #Purity_Louv_twice_list = [Purity_twice]*len(nodes_rest)
                        #plt.plot(Purity_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            #f.write('Purity_Louvain_twice: ' + str(round(Purity_twice,3)) + '\n')
                            f.write('Purity after top ' + str(k*100) + '% nodes: ' + str(round(Purity_List[0],3)) + '\n')
                            f.write('Max Purity: ' + str(round(max(Purity_List),3)) + '\n')

                        #plot preserve_ratio
                        plt.figure(preserve_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Preserv_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        
                        #plot balancedness
                        plt.figure(balance_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Balance_List,  color = colors[FR_type], label = str(FR_tp[FR_type])) 
                       
                        #plot InEdge Degrees
                        plt.figure(InEdge_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(InEdge_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                    
                plt.figure(NMI_vs_Purity)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3,2,idx+1)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                


                plt.figure(NMI_vs_Purity_whole)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]))
                plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=11, ncol=2)
                plt.tight_layout(rect=[0, 0, 1, 1])

                plt.figure(nmi_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)    
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(purity_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Purity')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(preserve_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Preserve Ratio')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(balance_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Balancedness')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(InEdge_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('InEdge Degrees')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()
    
    
    p = PdfPages(pdf_name)            
    fig_nums = plt.get_fignums() 
    figs = [plt.figure(n) for n in fig_nums] 
	
	# iterating over the numbers in list 
    for fig in figs: 
		# and saving the files 
        fig.savefig(p, format='pdf') 
	
	# close the object 
    p.close() 
    plt.close('all')
    
        
            

                                
                    

                    
                    



ALM 10068
Log transform done
(10068, 50)
Accuracy of  15 -NN graph is 0.701
151020
Number of nodes: 10068
NMI baseline Leiden: 0.587 Purity baseline Leiden: 0.281
 --- ALM (top 20.0%): FL + log(n) hops | res:  0.05
NMI of strong majority vote: 0.75 Purity of strong majority vote: 0.516
NMI of effective cluster accuracy: 0.618 Purity of effective cluster accuracy: 0.309
Number of nodes not included:  1409
NMI after top  20.0 % nodes: 0.848
Max NMI:  0.848
 --- ALM (top 20.0%): FL_ng + log(n) hops | res:  0.05
NMI of strong majority vote: 0.755 Purity of strong majority vote: 0.514
NMI of effective cluster accuracy: 0.597 Purity of effective cluster accuracy: 0.287
Number of nodes not included:  818
NMI after top  20.0 % nodes: 0.807
Max NMI:  0.807
 --- ALM (top 20.0%): FL_ng_prop + log(n) hops | res:  0.05
NMI of strong majority vote: 0.741 Purity of strong majority vote: 0.489
NMI of effective cluster accuracy: 0.557 Purity of effective cluster accuracy: 0.251
Number of nodes not incl

## Threshold Value vs Percentage

In [36]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import time 
import debug
from collections import defaultdict
import leidenalg as la 
import igraph as ig

res_=[] #0.26, 0.24 is best for Louvain, 3.3, 2.0
labels=[]
total_labels=[]
names = []

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['Zhengmix8eq']
#data_names = ['breast-cancer']
'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''
#data_names = ['Cora full', 'Citeseer', 'Eu core']
# FR_ord = ['Rd', 'Ord']
# FR_Recalc = ['FR_avg', 'FR_Re']
FR_tp = ['FL','FL_ng','FL_ng_prop','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    pdf_name = './Plots/'+data+'/Top_val_vs_%'+data+'.pdf'
    
    
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''
        #DO not recalculate Graph for non bulk RNA datasets
        if data not in ['mRNA','miRNA'] and survive==1:
            continue
        G, label= data_to_graph(data,survive)
        cluster_sizes = defaultdict(int)
        for node in G.nodes():
            cluster_sizes[label[node]]+=1
        #print('Selected Labels:',selected_labels_dict)
        #add self loop to nodes without outgoing edges
        #G_ is copy of G
        G_ = G.copy()
        # if (data in ['Cora full', 'Citeseer', 'Eu core']):
        #     for node in G.nodes():
        #         if not list(G.successors(node)):
        #             G_.remove_node(node)
        for node in G_.nodes():
            if not list(G_.successors(node)):
                #print(data, ' has a node without outgoing edges:',node)
                G_.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())
        

        data_directory = f'./Plots/{data}'
        file_path = f'{data_directory}/{data}.txt'
        os.makedirs(data_directory, exist_ok=True)
        with open(file_path, 'w') as f:
            f.write('------------------------------------------------------------------------\n')
        
        
            
        for Num_hop in [1]:
            plt.figure(figsize=(15,15)) # figure for NMI vs purity
            plt.figure(figsize=(15,8)) # figure for NMI vs Purity (whole)
            plt.figure(figsize=(15,15)) # figure for NMI
            plt.figure(figsize=(15,15)) # figure for Purity
            plt.figure(figsize=(15,15)) # figure for preserve_ratio
            plt.figure(figsize=(15,15)) # figure for balancedness
            plt.figure(figsize=(10,15)) # figure for InEdge Degrees
            #List of FR_values
            FR_precalculated = []
            for fr_type in range(4): 
                '''
                Calculate FlowRank for the whole graph
                '''
                if Num_hop==0:
                    node2FR = calc_FlowRank(G_, fr_type, 5)
                else:
                    node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
                FR_precalculated.append(node2FR)
            for th_idx, threshold_val in enumerate([-1,0.2,0.5,0.8]):
                for idx, res in enumerate([0.05,0.25, 0.5, 1, 1.5, 5]):
                #for idx, res in enumerate([0.05]):
                #for idx, res in enumerate([0.01,0.03,0.05,0.1,0.15,0.2]):
                    for Mod_type in [0]:
                        ''' 
                        Louvain on G (baseline)
                        '''
                        partition = debug.louvain_partitions(G, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        label_ = part_to_full_label(partition_,G.number_of_nodes())
                        NMI_baseline = NMI(label_, label)
                        Purity_baseline = met.purity_score(label, label_)
                        
                        '''
                        Leiden on G (Baseline)
                        '''
                        G_ig, mapping = networkx_to_igraph(G) # Mapping = igraph index to networkx index
                        partition = la.find_partition(G_ig, la.RBConfigurationVertexPartition, n_iterations=-1, resolution_parameter=res, seed=0)
                        label_ = part_to_full_label(partition, G.number_of_nodes(), mapping)
                        NMI_baseline_leiden = NMI(label, label_)
                        Purity_baseline_leiden = met.purity_score(label, label_)
                        print('NMI baseline Leiden:', round(NMI_baseline_leiden,3), 'Purity baseline Leiden:', round(Purity_baseline_leiden,3))
                        
                        for FR_type in range(4):
                            name =  data + ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops'
                            if FR_type==3:
                                name = data + ' (top ' + str(k*100) + '%): PageRank'
                            if data in ['mRNA','miRNA']:
                                name = data +  ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops' + ' + Survive: '+str(survive)
                                if FR_type==3:
                                    name = data + ' (top ' + str(k*100) + '%): PageRank' + ' + Survive: '+str(survive)
                            print(' ---',name,'| res: ',str(res))
                            
                            # Create the directory if it doesn't exist
                            os.makedirs(data_directory, exist_ok=True)
                            with open(file_path, 'a') as f:
                                f.write(' ---'+name+'| res: '+str(res)+'\n')
                                f.write('NMI baseline: ' + str(round(NMI_baseline,3)) + '\n')
                            

                            '''
                            Get induced subgraph of top k% nodes in FR value
                            '''
                            node2FR = FR_precalculated[FR_type]

                            # if data in ['Cora full', 'Citeseer', 'Eu core']:
                            #     for node in G.nodes():
                            #         if not list(G.successors(node)):
                            #             node2FR[node]=0

                            node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                            
                            if threshold_val==-1:
                                k_ = int(k*len(node_list))
                                top_nodes = node_list[:k_]
                                nodes_rest = node_list[k_:]
                            else:
                                top_nodes = [node for node in node_list if node2FR[node]>=threshold_val]
                                nodes_rest = [node for node in node_list if node2FR[node]<threshold_val]
                            print('Number of nodes in the subgraph:', len(top_nodes))
                            H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                            
                            '''
                            label counts for balancedness calculation
                            '''
                            selected_labels_dict = defaultdict(int)
                            for node in H.nodes():
                                selected_labels_dict[label[node]]+=1
                            
                            '''
                            Louvain on the subgraph H 
                            '''
                            
                            partition = debug.louvain_partitions(H, seed=0,resolution=res)
                            partition_ = deque(partition, maxlen=1).pop()
                            H_label = part_to_full_label(partition_,G.number_of_nodes())

                            '''
                            Add nodes to the subgraph by strong majority vote
                            '''
                            NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                            #The nmi and purity of strong majority vote
                            print('NMI of strong majority vote:', round(NMI_List[-1],3), 'Purity of strong majority vote:', round(Purity_List[-1],3))
                            
                            
                            '''
                            Plots
                            '''
                            NMI_vs_Purity, NMI_vs_Purity_whole, nmi_fig, purity_fig, preserve_fig, balance_fig, InEdge_fig = plt.get_fignums()[-7:]
                            max_dot_size = 100
                            #plot NMI vs Purity
                            markers = ['o','s','*','^','h','D'] #idx (resolution)
                            colors = ['b','g','y','r'] #FR_type
                            top_k = "top20%" if threshold_val==-1 else '>' + str(threshold_val)
                            plt.figure(NMI_vs_Purity)
                            plt.subplot(3,2,idx+1)
                            percent_nodes = (H.number_of_nodes() + len(NMI_List))*100 / G.number_of_nodes()
                            #plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size,label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '% nodes')
                            plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size,marker=markers[th_idx],label = str(FR_tp[FR_type])+ '|'+top_k+'|'+str(round(percent_nodes,2))+ '% nodes')
                            if FR_type==0 and th_idx==0:
                                plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size,label = 'Louv_baseline')
                                plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size,label = 'Leiden_baseline')
                            
                            #plot NMI vs Purity (whole)
                            plt.figure(NMI_vs_Purity_whole)
                            plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size, marker = markers[idx],label = str(FR_tp[FR_type])+ '|'+top_k+'|'+str(round(percent_nodes,2))+ '% nodes')
                            if FR_type==0 and th_idx==0:
                                plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size, marker = markers[idx],label = 'Louv_baseline|'+str(res))
                                plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size, marker = markers[idx],label = 'Leiden_baseline|'+str(res))
                            #plt.scatter(Purity_twice, NMI_twice, color = 'm', s = max_dot_size, marker = markers[idx],label = '2*Louv|'+str(res))

                            #plot NMI
                            plt.figure(nmi_fig)
                            plt.subplot(3, 2, idx+1)
                            plt.plot(NMI_List,  color = colors[FR_type], label = str(FR_tp[FR_type]) + '|'+top_k)
                            if FR_type==0 and th_idx==0: 
                                #Horizontal Line for Baselines
                                NMI_baseline_list = [NMI_baseline]*len(nodes_rest)
                                plt.plot(NMI_baseline_list, label = 'Louv_baseline')
                                NMI_baseline_leiden_list = [NMI_baseline_leiden]*len(nodes_rest)
                                plt.plot(NMI_baseline_leiden_list, label = 'Leiden_baseline')
                                
                            #NMI_Louv_twice_list = [NMI_twice]*len(nodes_rest)
                            #plt.plot(NMI_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                            
                            with open(file_path, 'a') as f:
                                f.write('# of nodes not included: ' + str(G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) + '\n')
                                #f.write('NMI_Louvain_twice: ' + str(round(NMI_twice,3)) + '\n')
                                f.write('NMI after top ' + str(k*100) + '% nodes: ' + str(round(NMI_List[0],3)) + '\n')
                                f.write('Max NMI: ' + str(round(max(NMI_List),3)) + '\n')
                            print('Number of nodes not included: ' , G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) 
                            print('NMI after top ', k*100, '% nodes:', round(NMI_List[0],3))
                            print('Max NMI: ', round(max(NMI_List),3))

                            #plot Purity
                            plt.figure(purity_fig)
                            plt.subplot(3, 2, idx+1)
                            plt.plot(Purity_List,  color = colors[FR_type], label = str(FR_tp[FR_type]) + '|'+top_k)
                            if FR_type==0 and th_idx==0: 
                                #Horizontal Line for Baselines
                                Purity_baseline_list = [Purity_baseline]*len(nodes_rest)
                                plt.plot(Purity_baseline_list, label = 'Louv_baseline')
                                Purity_baseline_leiden_list = [Purity_baseline_leiden]*len(nodes_rest)
                                plt.plot(Purity_baseline_leiden_list, label = 'Leiden_baseline')
                                
                        
                            #Purity_Louv_twice_list = [Purity_twice]*len(nodes_rest)
                            #plt.plot(Purity_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                            
                            with open(file_path, 'a') as f:
                                #f.write('Purity_Louvain_twice: ' + str(round(Purity_twice,3)) + '\n')
                                f.write('Purity after top ' + str(k*100) + '% nodes: ' + str(round(Purity_List[0],3)) + '\n')
                                f.write('Max Purity: ' + str(round(max(Purity_List),3)) + '\n')

                            #plot preserve_ratio
                            plt.figure(preserve_fig)
                            plt.subplot(3, 2, idx+1)
                            plt.plot(Preserv_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                            
                            #plot balancedness
                            plt.figure(balance_fig)
                            plt.subplot(3, 2, idx+1)
                            plt.plot(Balance_List,  color = colors[FR_type], label = str(FR_tp[FR_type])) 
                        
                            #plot InEdge Degrees
                            plt.figure(InEdge_fig)
                            plt.subplot(3, 2, idx+1)
                            plt.plot(InEdge_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        
                    plt.figure(NMI_vs_Purity)
                    plt.tight_layout(rect=[0,0,1,1])
                    plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=10)
                    plt.subplot(3,2,idx+1)
                    plt.xlabel('Purity')
                    plt.ylabel('NMI')
                    plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                    


                    plt.figure(NMI_vs_Purity_whole)
                    plt.xlabel('Purity')
                    plt.ylabel('NMI')
                    plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]))
                    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=6, ncol=2)
                    plt.tight_layout(rect=[0, 0, 1, 1])

                    plt.figure(nmi_fig)
                    plt.tight_layout(rect=[0,0,1,1])
                    plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                    plt.subplot(3, 2, idx+1)    
                    plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                    plt.ylabel('NMI')
                    plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                    

                    plt.figure(purity_fig)
                    plt.tight_layout(rect=[0,0,1,1])
                    plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                    plt.subplot(3, 2, idx+1)
                    plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                    plt.ylabel('Purity')
                    plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                    

                    plt.figure(preserve_fig)
                    plt.tight_layout(rect=[0,0,1,1])
                    plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                    plt.subplot(3, 2, idx+1)
                    plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                    plt.ylabel('Preserve Ratio')
                    plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                    

                    plt.figure(balance_fig)
                    plt.tight_layout(rect=[0,0,1,1])
                    plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                    plt.subplot(3, 2, idx+1)
                    plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                    plt.ylabel('Balancedness')
                    plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                    

                    plt.figure(InEdge_fig)
                    plt.tight_layout()
                    plt.subplot(3, 2, idx+1)
                    plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                    plt.ylabel('InEdge Degrees')
                    plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                    plt.legend()
        
    
    p = PdfPages(pdf_name)            
    fig_nums = plt.get_fignums() 
    #figs = [plt.figure(n) for n in fig_nums] 
    figs = [plt.figure(1), plt.figure(2)]
	# iterating over the numbers in list 
    for fig in figs: 
		# and saving the files 
        fig.savefig(p, format='pdf') 
	
	# close the object 
    p.close() 
    plt.close('all')
    
        
            

                                
                    

                    
                    



ALM 10068
Log transform done
(10068, 50)
Accuracy of  15 -NN graph is 0.701
151020
Number of nodes: 10068
NMI baseline Leiden: 0.588 Purity baseline Leiden: 0.282
 --- ALM (top 20.0%): FL + log(n) hops | res:  0.05
Number of nodes in the subgraph: 2013
NMI of strong majority vote: 0.772 Purity of strong majority vote: 0.514
Number of nodes not included:  1270
NMI after top  20.0 % nodes: 0.849
Max NMI:  0.849
 --- ALM (top 20.0%): FL_ng + log(n) hops | res:  0.05
Number of nodes in the subgraph: 2013
NMI of strong majority vote: 0.762 Purity of strong majority vote: 0.511
Number of nodes not included:  662
NMI after top  20.0 % nodes: 0.822
Max NMI:  0.822
 --- ALM (top 20.0%): FL_ng_prop + log(n) hops | res:  0.05
Number of nodes in the subgraph: 2013
NMI of strong majority vote: 0.717 Purity of strong majority vote: 0.471
Number of nodes not included:  949
NMI after top  20.0 % nodes: 0.779
Max NMI:  0.779
 --- ALM (top 20.0%): PageRank | res:  0.05
Number of nodes in the subgraph: 2

## Ablation (Exp2)

In [38]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import time 
import debug
from collections import defaultdict
import leidenalg as la 
import igraph as ig

res_=[] #0.26, 0.24 is best for Louvain, 3.3, 2.0
labels=[]
total_labels=[]
names = []

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['Zhengmix8eq']
#data_names = ['breast-cancer']
'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''
#data_names = ['Cora full', 'Citeseer', 'Eu core']
# FR_ord = ['Rd', 'Ord']
# FR_Recalc = ['FR_avg', 'FR_Re']
FR_tp = ['FL','FL_ng','FL_ng_prop','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    pdf_name = './Plots/'+data+'/Ablation'+data+'.pdf'
    
    
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''
        #DO not recalculate Graph for non bulk RNA datasets
        if data not in ['mRNA','miRNA'] and survive==1:
            continue
        G, label= data_to_graph(data,survive)
        cluster_sizes = defaultdict(int)
        for node in G.nodes():
            cluster_sizes[label[node]]+=1
        #print('Selected Labels:',selected_labels_dict)
        #add self loop to nodes without outgoing edges
        #G_ is copy of G
        G_ = G.copy()
        # if (data in ['Cora full', 'Citeseer', 'Eu core']):
        #     for node in G.nodes():
        #         if not list(G.successors(node)):
        #             G_.remove_node(node)
        for node in G_.nodes():
            if not list(G_.successors(node)):
                #print(data, ' has a node without outgoing edges:',node)
                G_.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())
        

        data_directory = f'./Plots/{data}'
        file_path = f'{data_directory}/{data}.txt'
        os.makedirs(data_directory, exist_ok=True)
        with open(file_path, 'w') as f:
            f.write('------------------------------------------------------------------------\n')
        
        
            
        for Num_hop in [1]:
            plt.figure(figsize=(15,15)) # figure for NMI vs purity
            plt.figure(figsize=(15,8)) # figure for NMI vs Purity (whole)
            plt.figure(figsize=(15,15)) # figure for NMI
            plt.figure(figsize=(15,15)) # figure for Purity
            plt.figure(figsize=(15,15)) # figure for preserve_ratio
            plt.figure(figsize=(15,15)) # figure for balancedness
            plt.figure(figsize=(10,15)) # figure for InEdge Degrees
            #List of FR_values
            FR_precalculated = []
            for fr_type in range(4): 
                '''
                Calculate FlowRank for the whole graph
                '''
                if Num_hop==0:
                    node2FR = calc_FlowRank(G_, fr_type, 5)
                else:
                    node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
                FR_precalculated.append(node2FR)

            for idx, res in enumerate([0.05,0.25, 0.5, 1, 1.5, 5]):
            #for idx, res in enumerate([0.05]):
            #for idx, res in enumerate([0.01,0.03,0.05,0.1,0.15,0.2]):
                for Mod_type in [0]:
                    ''' 
                    Louvain on G (baseline)
                    '''
                    partition = debug.louvain_partitions(G, seed=0,resolution=res)
                    partition_ = deque(partition, maxlen=1).pop()
                    label_ = part_to_full_label(partition_,G.number_of_nodes())
                    NMI_baseline = NMI(label_, label)
                    Purity_baseline = met.purity_score(label, label_)
                    
                    '''
                    Leiden on G (Baseline)
                    '''
                    G_ig, mapping = networkx_to_igraph(G) # Mapping = igraph index to networkx index
                    partition = la.find_partition(G_ig, la.RBConfigurationVertexPartition, n_iterations=-1, resolution_parameter=res, seed=0)
                    label_ = part_to_full_label(partition, G.number_of_nodes(), mapping)
                    NMI_baseline_leiden = NMI(label, label_)
                    Purity_baseline_leiden = met.purity_score(label, label_)
                    print('NMI baseline Leiden:', round(NMI_baseline_leiden,3), 'Purity baseline Leiden:', round(Purity_baseline_leiden,3))
                    
                    for FR_type in range(4):
                        name =  data + ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops'
                        if FR_type==3:
                            name = data + ' (top ' + str(k*100) + '%): PageRank'
                        if data in ['mRNA','miRNA']:
                            name = data +  ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops' + ' + Survive: '+str(survive)
                            if FR_type==3:
                                name = data + ' (top ' + str(k*100) + '%): PageRank' + ' + Survive: '+str(survive)
                        print(' ---',name,'| res: ',str(res))
                        
                        # Create the directory if it doesn't exist
                        os.makedirs(data_directory, exist_ok=True)
                        with open(file_path, 'a') as f:
                            f.write(' ---'+name+'| res: '+str(res)+'\n')
                            f.write('NMI baseline: ' + str(round(NMI_baseline,3)) + '\n')
                        

                        '''
                        Get induced subgraph of top k% nodes in FR value
                        '''
                        node2FR = FR_precalculated[FR_type]
                        
                        # if data in ['Cora full', 'Citeseer', 'Eu core']:
                        #     for node in G.nodes():
                        #         if not list(G.successors(node)):
                        #             node2FR[node]=0

                        node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                        
                        k_ = int(k*len(node_list))
                        top_nodes = node_list[:k_]
                        nodes_rest = node_list[k_:]
                        H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                        
                        '''
                        label counts for balancedness calculation
                        '''
                        selected_labels_dict = defaultdict(int)
                        for node in H.nodes():
                            selected_labels_dict[label[node]]+=1
                        
                        '''
                        Louvain on the subgraph H 
                        '''
                        
                        partition = debug.louvain_partitions(H, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        H_label = part_to_full_label(partition_,G.number_of_nodes())

                        '''
                        Add nodes to the subgraph by strong majority vote
                        '''
                        NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                        '''
                        Choose same amount of nodes from the list of nodes sorted by FR value and check Louvain NMI and purity
                        '''
                        node_num = len(NMI_List)+H.number_of_nodes()
                        H_ = getInducedSubgraph(G, node_list[:node_num])
                        partition = debug.louvain_partitions(H_, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        H_label_ = part_to_full_label(partition_,G.number_of_nodes())
                        H_label_compressed = []
                        label_compressed = []
                        for i in node_list[:node_num]:
                            if H_label_[i] != -1:
                                H_label_compressed.append(H_label_[i])
                                label_compressed.append(label[i])
                            
                        NMI_majority_vote_percentage_top_nodes = NMI(H_label_compressed, label_compressed)
                        Purity_majority_vote_percentage_top_nodes = met.purity_score(label_compressed, H_label_compressed)
                        print('NMI of top majority vote percentage nodes:', round(NMI_majority_vote_percentage_top_nodes,3), 
                              'Purity of top majority vote percentage nodes:', round(Purity_majority_vote_percentage_top_nodes,3))
                        
                        '''
                        Plots
                        '''
                        NMI_vs_Purity, NMI_vs_Purity_whole, nmi_fig, purity_fig, preserve_fig, balance_fig, InEdge_fig = plt.get_fignums()[-7:]
                        max_dot_size = 100
                        #plot NMI vs Purity
                        markers = ['o','s','*','^','h','D'] #idx (resolution)
                        colors = ['b','g','y','r'] #FR_type
                        plt.figure(NMI_vs_Purity)
                        plt.subplot(3,2,idx+1)
                        percent_nodes = (H.number_of_nodes() + len(NMI_List))*100 / G.number_of_nodes()
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size,label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '% nodes')
                        plt.scatter(Purity_majority_vote_percentage_top_nodes, NMI_majority_vote_percentage_top_nodes, s = percent_nodes/100*max_dot_size, edgecolors = colors[FR_type], facecolors='none', label = 'Abl|' + str(FR_tp[FR_type]) + '|'+str(round(percent_nodes,2))+ '% nodes')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size,label = 'Louv_baseline')
                            plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size,label = 'Leiden_baseline')
                        
                        #plot NMI vs Purity (whole)
                        plt.figure(NMI_vs_Purity_whole)
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size, marker = markers[idx],label = str(FR_tp[FR_type])+ '|'+str(res)+'|'+str(round(percent_nodes,2))+ '% nodes')
                        plt.scatter(Purity_majority_vote_percentage_top_nodes, NMI_majority_vote_percentage_top_nodes , edgecolors = colors[FR_type], facecolors='none', s = percent_nodes/100*max_dot_size, marker = markers[idx],label = 'Exp2|'+str(res)+'|'+str(round(percent_nodes,2))+ '% nodes')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size, marker = markers[idx],label = 'Louv_baseline|'+str(res))
                            plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size, marker = markers[idx],label = 'Leiden_baseline|'+str(res))
                        #plt.scatter(Purity_twice, NMI_twice, color = 'm', s = max_dot_size, marker = markers[idx],label = '2*Louv|'+str(res))

                        #plot NMI
                        plt.figure(nmi_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(NMI_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        NMI_louv_after_vote_list = [NMI_majority_vote_percentage_top_nodes]*len(nodes_rest)
                        plt.plot(NMI_louv_after_vote_list, label = 'Exp2|'+str(FR_tp_short[FR_type]))
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            NMI_baseline_list = [NMI_baseline]*len(nodes_rest)
                            plt.plot(NMI_baseline_list, label = 'Louv_baseline')
                            NMI_baseline_leiden_list = [NMI_baseline_leiden]*len(nodes_rest)
                            plt.plot(NMI_baseline_leiden_list, label = 'Leiden_baseline')
                            
                        #NMI_Louv_twice_list = [NMI_twice]*len(nodes_rest)
                        #plt.plot(NMI_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            f.write('# of nodes not included: ' + str(G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) + '\n')
                            #f.write('NMI_Louvain_twice: ' + str(round(NMI_twice,3)) + '\n')
                            f.write('NMI after top ' + str(k*100) + '% nodes: ' + str(round(NMI_List[0],3)) + '\n')
                            f.write('Max NMI: ' + str(round(max(NMI_List),3)) + '\n')
                        print('Number of nodes not included: ' , G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) 
                        print('NMI after top ', k*100, '% nodes:', round(NMI_List[0],3))
                        print('Max NMI: ', round(max(NMI_List),3))

                        #plot Purity
                        plt.figure(purity_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Purity_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        Purity_Louv_after_vote_list = [Purity_majority_vote_percentage_top_nodes]*len(nodes_rest)
                        plt.plot(Purity_Louv_after_vote_list, label = 'Exp2|'+str(FR_tp_short[FR_type]))
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            Purity_baseline_list = [Purity_baseline]*len(nodes_rest)
                            plt.plot(Purity_baseline_list, label = 'Louv_baseline')
                            Purity_baseline_leiden_list = [Purity_baseline_leiden]*len(nodes_rest)
                            plt.plot(Purity_baseline_leiden_list, label = 'Leiden_baseline')
                            
                    
                        #Purity_Louv_twice_list = [Purity_twice]*len(nodes_rest)
                        #plt.plot(Purity_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            #f.write('Purity_Louvain_twice: ' + str(round(Purity_twice,3)) + '\n')
                            f.write('Purity after top ' + str(k*100) + '% nodes: ' + str(round(Purity_List[0],3)) + '\n')
                            f.write('Max Purity: ' + str(round(max(Purity_List),3)) + '\n')

                        #plot preserve_ratio
                        plt.figure(preserve_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Preserv_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        
                        #plot balancedness
                        plt.figure(balance_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Balance_List,  color = colors[FR_type], label = str(FR_tp[FR_type])) 
                       
                        #plot InEdge Degrees
                        plt.figure(InEdge_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(InEdge_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                    
                plt.figure(NMI_vs_Purity)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3,2,idx+1)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                


                plt.figure(NMI_vs_Purity_whole)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]))
                plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=11, ncol=2)
                plt.tight_layout(rect=[0, 0, 1, 1])

                plt.figure(nmi_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)    
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(purity_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Purity')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(preserve_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Preserve Ratio')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(balance_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Balancedness')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(InEdge_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('InEdge Degrees')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()
    
    
    p = PdfPages(pdf_name)            
    fig_nums = plt.get_fignums() 
    figs = [plt.figure(n) for n in fig_nums] 
	
	# iterating over the numbers in list 
    for fig in figs: 
		# and saving the files 
        fig.savefig(p, format='pdf') 
	
	# close the object 
    p.close() 
    plt.close('all')
    
        
            

                                
                    

                    
                    



NMI of top majority vote percentage nodes: 0.416 Purity of top majority vote percentage nodes: 0.629
Number of nodes not included:  5445
NMI after top  20.0 % nodes: 0.467
Max NMI:  0.475
 --- Zheng (top 20.0%): PageRank | res:  1.5
NMI of top majority vote percentage nodes: 0.42 Purity of top majority vote percentage nodes: 0.629
Number of nodes not included:  5224
NMI after top  20.0 % nodes: 0.433
Max NMI:  0.455
NMI baseline Leiden: 0.346 Purity baseline Leiden: 0.643
 --- Zheng (top 20.0%): FL + log(n) hops | res:  5
NMI of top majority vote percentage nodes: 0.379 Purity of top majority vote percentage nodes: 0.664
Number of nodes not included:  19810
NMI after top  20.0 % nodes: 0.455
Max NMI:  0.458
 --- Zheng (top 20.0%): FL_ng + log(n) hops | res:  5
NMI of top majority vote percentage nodes: 0.372 Purity of top majority vote percentage nodes: 0.662
Number of nodes not included:  18213
NMI after top  20.0 % nodes: 0.438
Max NMI:  0.447
 --- Zheng (top 20.0%): FL_ng_prop + log

## Louv + Vote + Louv 

In [18]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

res_=[] #0.26, 0.24 is best for Louvain, 3.3, 2.0
labels=[]
total_labels=[]
names = []

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''
#data_names = ['Cora full', 'Citeseer', 'Eu core']
# FR_ord = ['Rd', 'Ord']
# FR_Recalc = ['FR_avg', 'FR_Re']
FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    pdf_name = './Plots/'+data+'/(Louv_Vote)x2'+data+'.pdf'
    
    
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''
        #DO not recalculate Graph for non bulk RNA datasets
        if data not in ['mRNA','miRNA'] and survive==1:
            continue
        G, label= data_to_graph(data,survive)
        True_num_of_clusters = len(set(label))
        cluster_sizes = defaultdict(int)
        for node in G.nodes():
            cluster_sizes[label[node]]+=1
        #print('Selected Labels:',selected_labels_dict)
        #add self loop to nodes without outgoing edges
        #G_ is copy of G
        G_ = G.copy()
        # if (data in ['Cora full', 'Citeseer', 'Eu core']):
        #     for node in G.nodes():
        #         if not list(G.successors(node)):
        #             G_.remove_node(node)
        for node in G_.nodes():
            if not list(G_.successors(node)):
                #print(data, ' has a node without outgoing edges:',node)
                G_.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())
        

        data_directory = f'./Plots/{data}'
        file_path = f'{data_directory}/{data}.txt'
        os.makedirs(data_directory, exist_ok=True)
        with open(file_path, 'w') as f:
            f.write('------------------------------------------------------------------------\n')
        
        
            
        for Num_hop in [1]:
            plt.figure(figsize=(15,15)) # figure for NMI vs purity
            plt.figure(figsize=(15,8)) # figure for NMI vs Purity (whole)
            plt.figure(figsize=(15,15)) # figure for NMI
            plt.figure(figsize=(15,15)) # figure for Purity
            plt.figure(figsize=(15,15)) # figure for preserve_ratio
            plt.figure(figsize=(15,15)) # figure for balancedness
            plt.figure(figsize=(10,15)) # figure for InEdge Degrees
            #List of FR_values
            FR_precalculated = []
            for fr_type in range(4): 
                '''
                Calculate FlowRank for the whole graph
                '''
                if Num_hop==0:
                    node2FR = calc_FlowRank(G_, fr_type, 5)
                else:
                    node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
                FR_precalculated.append(node2FR)

            for idx, res in enumerate([0.05,0.25, 0.5, 1, 1.5, 5]):
            #for idx, res in enumerate([0.05]):
            #for idx, res in enumerate([0.01,0.03,0.05,0.1,0.15,0.2]):
                for Mod_type in [0]:
                    ''' 
                    Louvain on G (baseline)
                    '''
                    partition = debug.louvain_partitions(G, seed=0,resolution=res)
                    partition_ = deque(partition, maxlen=1).pop()
                    label_ = part_to_full_label(partition_,G.number_of_nodes())
                    NMI_baseline = NMI(label_, label)
                    Purity_baseline = met.purity_score(label, label_)
                    #Num of cluster 
                    Num_Cluster_Louv_on_G = len(set(label_))
                    '''
                    Leiden on G (Baseline)
                    '''
                    G_ig, mapping = networkx_to_igraph(G) # Mapping = igraph index to networkx index
                    partition = la.find_partition(G_ig, la.RBConfigurationVertexPartition, n_iterations=-1, resolution_parameter=res, seed=0)
                    label_ = part_to_full_label(partition, G.number_of_nodes(), mapping)
                    NMI_baseline_leiden = NMI(label, label_)
                    Purity_baseline_leiden = met.purity_score(label, label_)
                    print('NMI baseline Leiden:', round(NMI_baseline_leiden,3), 'Purity baseline Leiden:', round(Purity_baseline_leiden,3))
                    Num_Cluster_Leiden_on_G = len(set(label_))
                    for FR_type in range(4):
                        name =  data + ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops'
                        if FR_type==3:
                            name = data + ' (top ' + str(k*100) + '%): PageRank'
                        if data in ['mRNA','miRNA']:
                            name = data +  ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops' + ' + Survive: '+str(survive)
                            if FR_type==3:
                                name = data + ' (top ' + str(k*100) + '%): PageRank' + ' + Survive: '+str(survive)
                        print(' ---',name,'| res: ',str(res))
                        
                        # Create the directory if it doesn't exist
                        os.makedirs(data_directory, exist_ok=True)
                        with open(file_path, 'a') as f:
                            f.write(' ---'+name+'| res: '+str(res)+'\n')
                            f.write('NMI baseline: ' + str(round(NMI_baseline,3)) + '\n')
                        

                        '''
                        Get induced subgraph of top k% nodes in FR value
                        '''
                        node2FR = FR_precalculated[FR_type]
                        
                        # if data in ['Cora full', 'Citeseer', 'Eu core']:
                        #     for node in G.nodes():
                        #         if not list(G.successors(node)):
                        #             node2FR[node]=0

                        node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                        
                        k_ = int(k*len(node_list))
                        top_nodes = node_list[:k_]
                        nodes_rest = node_list[k_:]
                        H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                        
                        '''
                        label counts for balancedness calculation
                        '''
                        selected_labels_dict = defaultdict(int)
                        for node in H.nodes():
                            selected_labels_dict[label[node]]+=1
                        
                        '''
                        Louvain on the subgraph H 
                        '''
                        
                        partition = debug.louvain_partitions(H, seed=0,resolution=res)
                        partition_ = deque(partition, maxlen=1).pop()
                        H_label = part_to_full_label(partition_,G.number_of_nodes())
                        
                        Num_Cluster_Louv_on_H = len({x for x in set(H_label) if x != -1})
                        '''
                        Add nodes to the subgraph by strong majority vote
                        '''
                        NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                        '''
                        Louv again on H+votes subgraph with pre-labels
                        '''
                        init_partition = label_to_partition(H_label,only_labeled_nodes = 1)
                        selected_node_list = [node_num_ for node_num_, node_label_ in enumerate(H_label) if node_label_ != -1]
                        H2 = getInducedSubgraph(G, selected_node_list)
                        partition2 = Cust.louvain_partitions(H2, seed=0,resolution=res, init_part=init_partition)  
                        partition2_ = deque(partition2, maxlen=1).pop()
                        H_label2 = part_to_full_label(partition2_,G.number_of_nodes())
                        label_compressed2, H_label_compressed2 = get_compressed_labels(label,H_label2, selected_node_list)
                        NMI_louv_vote_louv = NMI(H_label_compressed2, label_compressed2)
                        Purity_louv_vote_louv = met.purity_score(label_compressed2, H_label_compressed2)
                        Num_Cluster_Louv_on_H2 = len({x for x in set(H_label2) if x != -1})
                        '''
                        Louv + vote + Louv + vote
                        '''
                        nodes_rest2 = [node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ == -1]
                        NMI_List2, Purity_List2, Balance_List2, Preserv_List2, InEdge_List2 = merge_by_vote(selected_node_list, nodes_rest2, H_label2, G, label, selected_labels_dict, cluster_sizes)
                        selected_node_list2 = [node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ != -1]
                        '''
                        Plots
                        '''
                        NMI_vs_Purity, NMI_vs_Purity_whole, nmi_fig, purity_fig, preserve_fig, balance_fig, InEdge_fig = plt.get_fignums()[-7:]
                        max_dot_size = 100
                        #plot NMI vs Purity
                        markers = ['o','s','*','^','h','D'] #idx (resolution)
                        colors = ['b','g','y','r'] #FR_type
                        plt.figure(NMI_vs_Purity)
                        plt.subplot(3,2,idx+1)
                        percent_nodes = len(selected_node_list)*100 / G.number_of_nodes()
                        percent_nodes2 = len(selected_node_list2)*100 / G.number_of_nodes()
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size,label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '%' + '|#Com:'+str(Num_Cluster_Louv_on_H))
                        plt.scatter(Purity_louv_vote_louv, NMI_louv_vote_louv, s = percent_nodes/100*max_dot_size, edgecolors = colors[FR_type], facecolors='none', label = 'Lv+Vote+Lv|' + str(FR_tp[FR_type]) + '|'+str(round(percent_nodes,2))+ '%'+ '|#Com:'+str(Num_Cluster_Louv_on_H2))
                        plt.scatter(Purity_List2[-1], NMI_List2[-1], edgecolors = colors[FR_type], facecolors='none', s = percent_nodes2/100*max_dot_size,marker = 's', label = '(Lv+Vote)x2|'+str(FR_tp[FR_type])+ '|'+str(round(percent_nodes2,2))+ '%'+ '|#Com:'+str(Num_Cluster_Louv_on_H2))
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size,label = 'Louv_baseline'+ '|#Com:'+str(Num_Cluster_Louv_on_G))
                            plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size,label = 'Leiden_baseline'+ '|#Com:'+str(Num_Cluster_Leiden_on_G))
                        
                        #plot NMI vs Purity (whole)
                        plt.figure(NMI_vs_Purity_whole)
                        plt.scatter(Purity_List[-1], NMI_List[-1], color = colors[FR_type], s = percent_nodes/100*max_dot_size, marker = markers[idx],label = str(FR_tp[FR_type])+ '|'+str(res)+'|'+str(round(percent_nodes,2))+ '%')
                        plt.scatter(Purity_louv_vote_louv, NMI_louv_vote_louv, s = percent_nodes/100*max_dot_size, edgecolors = colors[FR_type], facecolors='none', marker = markers[idx],label = 'Lv+Vote+Lv|' + str(FR_tp[FR_type]) + '|'+str(res)+'|'+str(round(percent_nodes,2))+ '%')
                        if FR_type==0:
                            plt.scatter(Purity_baseline, NMI_baseline, color = 'k', s = max_dot_size, marker = markers[idx],label = 'Louv_baseline|'+str(res))
                            plt.scatter(Purity_baseline_leiden, NMI_baseline_leiden, color = 'm', s = max_dot_size, marker = markers[idx],label = 'Leiden_baseline|'+str(res))
                        #plt.scatter(Purity_twice, NMI_twice, color = 'm', s = max_dot_size, marker = markers[idx],label = '2*Louv|'+str(res))

                        #plot NMI
                        plt.figure(nmi_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(NMI_List,  color = colors[FR_type], label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '% nodes')
                        NMI_List2 = [NMI_List2[0]]*len(NMI_List)+NMI_List2
                        plt.plot(NMI_List2,  color = colors[FR_type], label = str(FR_tp[FR_type])+'x2'+'|'+str(round(percent_nodes2,2))+ '% nodes')
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            NMI_baseline_list = [NMI_baseline]*len(nodes_rest)
                            plt.plot(NMI_baseline_list, label = 'Louv_baseline')
                            NMI_baseline_leiden_list = [NMI_baseline_leiden]*len(nodes_rest)
                            plt.plot(NMI_baseline_leiden_list, label = 'Leiden_baseline')
                            
                        #NMI_Louv_twice_list = [NMI_twice]*len(nodes_rest)
                        #plt.plot(NMI_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            f.write('# of nodes not included: ' + str(G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) + '\n')
                            #f.write('NMI_Louvain_twice: ' + str(round(NMI_twice,3)) + '\n')
                            f.write('NMI after top ' + str(k*100) + '% nodes: ' + str(round(NMI_List[0],3)) + '\n')
                            f.write('Max NMI: ' + str(round(max(NMI_List),3)) + '\n')
                        print('Number of nodes not included: ' , G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) 
                        print('NMI after top ', k*100, '% nodes:', round(NMI_List[0],3))
                        print('Max NMI: ', round(max(NMI_List),3))

                        #plot Purity
                        plt.figure(purity_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Purity_List,  color = colors[FR_type], label = str(FR_tp[FR_type])+ '|'+str(round(percent_nodes,2))+ '% nodes')
                        Purity_List2 = [Purity_List2[0]]*len(Purity_List)+Purity_List2
                        plt.plot(Purity_List2,  color = colors[FR_type], label = str(FR_tp[FR_type])+'x2'+'|'+str(round(percent_nodes2,2))+ '% nodes')
                        if FR_type==0: 
                            #Horizontal Line for Baselines
                            Purity_baseline_list = [Purity_baseline]*len(nodes_rest)
                            plt.plot(Purity_baseline_list, label = 'Louv_baseline')
                            Purity_baseline_leiden_list = [Purity_baseline_leiden]*len(nodes_rest)
                            plt.plot(Purity_baseline_leiden_list, label = 'Leiden_baseline')
                            
                    
                        #Purity_Louv_twice_list = [Purity_twice]*len(nodes_rest)
                        #plt.plot(Purity_Louv_twice_list, label = '2*Louv|'+str(FR_tp_short[FR_type]))
                        
                        with open(file_path, 'a') as f:
                            #f.write('Purity_Louvain_twice: ' + str(round(Purity_twice,3)) + '\n')
                            f.write('Purity after top ' + str(k*100) + '% nodes: ' + str(round(Purity_List[0],3)) + '\n')
                            f.write('Max Purity: ' + str(round(max(Purity_List),3)) + '\n')

                        #plot preserve_ratio
                        plt.figure(preserve_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Preserv_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        Preserv_List2 = [Preserv_List2[0]]*len(Preserv_List)+Preserv_List2
                        plt.plot(Preserv_List2,  color = colors[FR_type], label = str(FR_tp[FR_type])+'x2')

                        #plot balancedness
                        plt.figure(balance_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(Balance_List,  color = colors[FR_type], label = str(FR_tp[FR_type])) 
                        Balance_List2 = [Balance_List2[0]]*len(Balance_List)+Balance_List2
                        plt.plot(Balance_List2,  color = colors[FR_type], label = str(FR_tp[FR_type])+'x2')

                        #plot InEdge Degrees
                        plt.figure(InEdge_fig)
                        plt.subplot(3, 2, idx+1)
                        plt.plot(InEdge_List,  color = colors[FR_type], label = str(FR_tp[FR_type]))
                        InEdge_List2 = [InEdge_List2[0]]*len(InEdge_List)+InEdge_List2
                        plt.plot(InEdge_List2,  color = colors[FR_type], label = str(FR_tp[FR_type])+'x2')
                    
                plt.figure(NMI_vs_Purity)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=9)
                plt.subplot(3,2,idx+1)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|#OfComm: '+str(True_num_of_clusters))
                


                plt.figure(NMI_vs_Purity_whole)
                plt.xlabel('Purity')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]))
                plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=11, ncol=2)
                plt.tight_layout(rect=[0, 0, 1, 1])

                plt.figure(nmi_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)    
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('NMI')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(purity_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Purity')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(preserve_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Preserve Ratio')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(balance_fig)
                plt.tight_layout(rect=[0,0,1,1])
                plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5), fontsize=11)
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('Balancedness')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                

                plt.figure(InEdge_fig)
                plt.tight_layout()
                plt.subplot(3, 2, idx+1)
                plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                plt.ylabel('InEdge Degrees')
                plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|')
                plt.legend()
    
    
    p = PdfPages(pdf_name)            
    fig_nums = plt.get_fignums() 
    figs = [plt.figure(n) for n in fig_nums] 
	
	# iterating over the numbers in list 
    for fig in figs: 
		# and saving the files 
        fig.savefig(p, format='pdf') 
	
	# close the object 
    p.close() 
    plt.close('all')
    
        
            

                                
                    

                    
                    



Zhengmix8eq 3994
Log transform done
(3994, 50)
Accuracy of  15 -NN graph is 0.790
59910
Number of nodes: 3994
NMI baseline Leiden: 0.69 Purity baseline Leiden: 0.546
 --- Zhengmix8eq (top 20.0%): FL + log(n) hops | res:  0.05
# of Partition:  8
# of Partition:  4
# of Partition:  4
Number of nodes not included:  145
NMI after top  20.0 % nodes: 0.825
Max NMI:  0.841
 --- Zhengmix8eq (top 20.0%): FL_ng + log(n) hops | res:  0.05
# of Partition:  10
# of Partition:  4
# of Partition:  4
Number of nodes not included:  144
NMI after top  20.0 % nodes: 0.735
Max NMI:  0.77
 --- Zhengmix8eq (top 20.0%): FL_ng_pr + log(n) hops | res:  0.05
# of Partition:  10
# of Partition:  4
# of Partition:  4
Number of nodes not included:  46
NMI after top  20.0 % nodes: 0.687
Max NMI:  0.721
 --- Zhengmix8eq (top 20.0%): PageRank | res:  0.05
# of Partition:  6
# of Partition:  4
# of Partition:  4
Number of nodes not included:  99
NMI after top  20.0 % nodes: 0.715
Max NMI:  0.721
NMI baseline Leiden: 0

C:\Users\MSI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


20 19
Number of nodes: 10909
NMI baseline Leiden: 0.299 Purity baseline Leiden: 0.352
 --- 20NewsGroups_tfdif (top 20.0%): FL + log(n) hops | res:  0.05
# of Partition:  349
# of Partition:  307
# of Partition:  306
# of Partition:  306
Number of nodes not included:  6155
NMI after top  20.0 % nodes: 0.59
Max NMI:  0.59
 --- 20NewsGroups_tfdif (top 20.0%): FL_ng + log(n) hops | res:  0.05
# of Partition:  366
# of Partition:  334
# of Partition:  334
Number of nodes not included:  6437
NMI after top  20.0 % nodes: 0.602
Max NMI:  0.602
 --- 20NewsGroups_tfdif (top 20.0%): FL_ng_pr + log(n) hops | res:  0.05
# of Partition:  477
# of Partition:  431
# of Partition:  430
# of Partition:  430
Number of nodes not included:  6662
NMI after top  20.0 % nodes: 0.603
Max NMI:  0.603
 --- 20NewsGroups_tfdif (top 20.0%): PageRank | res:  0.05
# of Partition:  188
# of Partition:  155
# of Partition:  154
# of Partition:  154
Number of nodes not included:  5614
NMI after top  20.0 % nodes: 0.49
M

i:\내 드라이브\backup\document\USC\Research\MCPC\Code\Codes\New_Cleaned\FlowRank_General.py:424: RuntimeWarning: invalid value encountered in scalar divide
  rank[v]=v_cover[v]/sc


NMI baseline Leiden: 0.193 Purity baseline Leiden: 0.086
 --- Cora full (top 20.0%): FL + log(n) hops | res:  0.05
# of Partition:  4633
# of Partition:  4580
# of Partition:  4580
Number of nodes not included:  16627
NMI after top  20.0 % nodes: 0.643
Max NMI:  0.643
 --- Cora full (top 20.0%): FL_ng + log(n) hops | res:  0.05
# of Partition:  371
# of Partition:  242
# of Partition:  241
# of Partition:  241
Number of nodes not included:  16934
NMI after top  20.0 % nodes: 0.352
Max NMI:  0.352
 --- Cora full (top 20.0%): FL_ng_pr + log(n) hops | res:  0.05
# of Partition:  315
# of Partition:  175
# of Partition:  173
# of Partition:  173
Number of nodes not included:  16905
NMI after top  20.0 % nodes: 0.335
Max NMI:  0.335
 --- Cora full (top 20.0%): PageRank | res:  0.05
# of Partition:  4521
# of Partition:  4328
# of Partition:  4324
# of Partition:  4324
Number of nodes not included:  14712
NMI after top  20.0 % nodes: 0.643
Max NMI:  0.643
NMI baseline Leiden: 0.364 Purity ba

i:\내 드라이브\backup\document\USC\Research\MCPC\Code\Codes\New_Cleaned\FlowRank_General.py:424: RuntimeWarning: invalid value encountered in scalar divide
  rank[v]=v_cover[v]/sc


NMI baseline Leiden: 0.397 Purity baseline Leiden: 0.65
 --- Citeseer (top 20.0%): FL + log(n) hops | res:  0.05
# of Partition:  424
# of Partition:  398
# of Partition:  397
# of Partition:  397
Number of nodes not included:  1247
NMI after top  20.0 % nodes: 0.459
Max NMI:  0.459
 --- Citeseer (top 20.0%): FL_ng + log(n) hops | res:  0.05
# of Partition:  63
# of Partition:  61
# of Partition:  61
Number of nodes not included:  1643
NMI after top  20.0 % nodes: 0.428
Max NMI:  0.428
 --- Citeseer (top 20.0%): FL_ng_pr + log(n) hops | res:  0.05
# of Partition:  69
# of Partition:  67
# of Partition:  67
Number of nodes not included:  1633
NMI after top  20.0 % nodes: 0.425
Max NMI:  0.425
 --- Citeseer (top 20.0%): PageRank | res:  0.05
# of Partition:  398
# of Partition:  353
# of Partition:  349
# of Partition:  349
Number of nodes not included:  928
NMI after top  20.0 % nodes: 0.453
Max NMI:  0.453
NMI baseline Leiden: 0.364 Purity baseline Leiden: 0.681
 --- Citeseer (top 20.0